In [1]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt").to('mps')

/Users/amaljoe/Desktop/Workspace/IITB/NLP/OCR_with_LLMs/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
model.train(data="train_model.yaml", epochs=100)

In [22]:
model.load("runs/detect/train103/weights/last.pt")

Transferred 79/175 items from pretrained weights


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): ModuleList(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
    

In [86]:
import cv2
import matplotlib.pyplot as plt

image_path = "yolo_dataset/train/b06-045.png"

# Make predictions
model = YOLO("runs/detect/train103/weights/best.pt").to('mps')
results = model(image_path, save=True)
results[0].save_dir + results[0].path[results[0].path.rindex('/'):]
# Display results


image 1/1 /Users/amaljoe/Desktop/Workspace/IITB/NLP/OCR_with_LLMs/yolo_dataset/train/b06-045.png: 640x448 12 handwritten_lines, 40.5ms
Speed: 3.6ms preprocess, 40.5ms inference, 9.3ms postprocess per image at shape (1, 3, 640, 448)
Results saved to /Users/amaljoe/Desktop/Workspace/IITB/NLP/OCR_with_LLMs/runs/detect/predict17


'/Users/amaljoe/Desktop/Workspace/IITB/NLP/OCR_with_LLMs/runs/detect/predict17/b06-045.png'

In [74]:
from PIL import Image

patches = []
ys = []
for box in sorted(results[0].boxes, key=lambda x: x.xywh[0][1]):
    image = Image.open(image_path).convert("RGB")
    x_center, y_center, w, h  = box.xywh[0].cpu().numpy()
    x, y = x_center - w / 2, y_center - h / 2
    print(y)
    cropped_image = image.crop((x, y, x + w, y + h))
    patches.append(cropped_image)
    ys.append(y)

690.7214050292969
853.6131286621094
877.7685852050781
1041.1939697265625
1063.7310180664062
1208.0228881835938
1235.5355224609375
1388.7977905273438
1418.0924682617188
1588.8281860351562
1751.4537353515625
1945.95654296875


In [78]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import numpy as np
import pandas as pd
from nltk.translate import bleu_score
from nltk.translate.bleu_score import SmoothingFunction
import torch

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
trocr_model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten').to(device)
trocr_model.config.num_beams = 2
print(f'TrOCR Model loaded on {device}')

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 1024,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decod

Model loaded on mps


In [80]:
CONFIDENCE_THRESHOLD = 0.72
BLEU_THRESHOLD = 0.5
def inference(cropped_images, y, debug=False, return_texts='final'):
    def get_model_output(images):
        pixel_values = processor(images=images, return_tensors="pt").pixel_values.to(device)
        output = trocr_model.generate(pixel_values, return_dict_in_generate=True, output_scores=True, max_new_tokens=30)
        generated_texts = processor.batch_decode(output.sequences, skip_special_tokens=True)
        return generated_texts, output.sequences_scores

    def post_process_texts(generated_texts):
        for i in range(len(generated_texts)):
            if len(generated_texts[i]) > 2 and generated_texts[i][:2] == '# ':
                generated_texts[i] = generated_texts[i][2:]
        return generated_texts

    def get_qualified_texts(generated_texts, scores, y):
        qualified_texts = []
        for text, score, y_i in zip(generated_texts, scores, y):
            if score > CONFIDENCE_THRESHOLD:
                qualified_texts.append({
                    'text': text,
                    'score': score,
                    'y': y_i
                })
        return qualified_texts

    def get_adjacent_bleu_scores(qualified_texts):
        def get_bleu_score(hypothesis, references):
            weights = [0.5, 0.5]
            smoothing = SmoothingFunction()
            return bleu_score.sentence_bleu(references, hypothesis, weights=weights, smoothing_function=smoothing.method1)

        for i in range(len(qualified_texts)):
            hyp = qualified_texts[i]['text'].split()
            bleu = 0
            if i < len(qualified_texts) - 1:
                ref = qualified_texts[i + 1]['text'].split()
                bleu = get_bleu_score(hyp, [ref])
            qualified_texts[i]['bleu'] = bleu
        return qualified_texts

    def remove_overlapping_texts(qualified_texts):
        final_texts = []
        new = True
        for i in range(len(qualified_texts)):
            if new:
                final_texts.append(qualified_texts[i])
            else:
                if final_texts[-1]['score'] < qualified_texts[i]['score']:
                    final_texts[-1] = qualified_texts[i]
            new = qualified_texts[i]['bleu'] < BLEU_THRESHOLD
        return final_texts



    if debug:
        print('Number of cropped images:', len(cropped_images))
    generated_texts, scores = get_model_output(cropped_images)
    normalised_scores = np.exp(scores.to('cpu').numpy())
    if return_texts == 'generated':
        return pd.DataFrame({
            'text': generated_texts,
            'score': normalised_scores,
            'y': y
        })
    generated_texts = post_process_texts(generated_texts)
    if return_texts == 'post_processed':
        return pd.DataFrame({
            'text': generated_texts,
            'score': normalised_scores,
            'y': y
        })
    qualified_texts = get_qualified_texts(generated_texts, normalised_scores, y)
    if return_texts == 'qualified':
        return pd.DataFrame(qualified_texts)
    qualified_texts = get_adjacent_bleu_scores(qualified_texts)
    if return_texts == 'qualified_with_bleu':
        return pd.DataFrame(qualified_texts)
    final_texts = remove_overlapping_texts(qualified_texts)
    final_texts_df = pd.DataFrame(final_texts, columns=['text', 'score', 'y'])
    return final_texts_df

inference(patches, ys, debug=False, return_texts='final')

,text,score,y
0,For many years my union had to present its views,0.935786,690.721405
1,to the two major conferences of the movement and,0.997024,877.768585
2,"take a licking . For us , unity meant accepting",0.896918,1063.731018
3,unpalatable majority decisions as binding on the,0.895445,1208.022888
4,environment and having a go next time . Alongside,0.836036,1418.092468
5,"many others , we conducted our fight by putting",0.910007,1588.828186
6,down # unambiguous alternatives to,0.958086,1751.453735
7,official policy and seeking to win majorities .,0.912684,1945.956543


In [62]:
patches

[<PIL.Image.Image image mode=L size=2400x160>,
 <PIL.Image.Image image mode=L size=2418x140>,
 <PIL.Image.Image image mode=L size=2409x129>,
 <PIL.Image.Image image mode=L size=2402x146>,
 <PIL.Image.Image image mode=L size=2407x146>,
 <PIL.Image.Image image mode=L size=2334x145>,
 <PIL.Image.Image image mode=L size=2407x151>,
 <PIL.Image.Image image mode=L size=2404x149>]